# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction JSON Dataset to CSV Dataset Conversion

## UPDATE: There was a bug in previous version, in train set all measurement values were flattened to the first 2400\*68 entries followed by 2400\*39 nan. This is updated to flatten correctly with 68 entries and 39 nan for each id.

I think that for EDA, a flat file based representation would be good. Hence I am converting the json hierarchical dataset of the competition to CSV format popular among Kagglers.

## References: My public notebooks for OpenVaccine

* [Flatten JSON Data](https://www.kaggle.com/arunprathap/openvaccine-flatten-json-data)

* [Feature Correlation, Importance](https://www.kaggle.com/arunprathap/openvaccine-feature-correlation-importance)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from pandas.io.json import json_normalize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Read train and test json

In [ ]:
train_full = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
train_full.shape

In [ ]:
test_full = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
test_full.shape

In [ ]:
train_full.head().T

In [ ]:
test_full.head().T

Not so intuitive, right. Lets see if we can flatten it.

In [ ]:
train_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/train.json', 'r')]
test_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/test.json', 'r')]

In [ ]:
def preprocess_json(input_json, is_train):
    
    for index,json_ in enumerate(input_json):
        json_['step'] = list(range(json_['seq_length']))
        json_['sequence'] = list(json_['sequence'])
        json_['structure'] = list(json_['structure'])
        json_['predicted_loop_type'] = list(json_['predicted_loop_type'])
        if os.path.exists('../input/stanford-covid-vaccine/bpps/'+json_['id']+'.npy'):
            np_array = np.load('../input/stanford-covid-vaccine/bpps/'+json_['id']+'.npy')
            json_['unpaired_probability'] = list(1-sum(np_array))
        else:
            print('bpps not found')
        if is_train:
            append_list = [np.nan for i in range(39)]
            json_['reactivity'].extend(append_list)
            json_['reactivity_error'].extend(append_list)
            json_['deg_Mg_pH10'].extend(append_list)
            json_['deg_error_Mg_pH10'].extend(append_list)
            json_['deg_pH10'].extend(append_list)
            json_['deg_error_pH10'].extend(append_list)
            json_['deg_Mg_50C'].extend(append_list)
            json_['deg_error_Mg_50C'].extend(append_list)
            json_['deg_50C'].extend(append_list)
            json_['deg_error_50C'].extend(append_list)
                
preprocess_json(train_json, True)
preprocess_json(test_json, False)

In [ ]:
def process_json(input_json, is_train):
    if is_train:
        train = pd.json_normalize(data = input_json, 
                                record_path ='step',  
                                meta =['id','seq_length','seq_scored','SN_filter','signal_to_noise']) 
    else:
        train = pd.json_normalize(data = input_json, 
                                record_path ='step',  
                                meta =['id','seq_length','seq_scored']) 
    train.rename(columns={0:'step'}, inplace=True)
    train['unpaired_probability'] = pd.json_normalize(data = input_json, 
                                record_path ='unpaired_probability'
                                            )
    train['sequence'] = pd.json_normalize(data = input_json, 
                                record_path ='sequence'
                                            )
    train['structure'] = pd.json_normalize(data = input_json, 
                                record_path ='structure'
                                            )
    train['predicted_loop_type'] = pd.json_normalize(data = input_json, 
                                record_path ='predicted_loop_type'
                                            )
    if is_train:
        train['reactivity'] = pd.json_normalize(data = input_json, 
                                        record_path ='reactivity'
                                                    )
        train['reactivity_error'] = pd.json_normalize(data = input_json, 
                                        record_path ='reactivity_error'
                                                    )
        train['deg_Mg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_Mg_pH10'
                                                    )
        train['deg_error_Mg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_Mg_pH10'
                                                    )
        train['deg_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_pH10',
                                                    )
        train['deg_error_pH10'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_pH10',
                                                    )
        train['deg_Mg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_Mg_50C',
                                                    )
        train['deg_error_Mg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_Mg_50C',
                                                    )
        train['deg_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_50C',
                                                    )
        train['deg_error_50C'] = pd.json_normalize(data = input_json, 
                                        record_path ='deg_error_50C',
                                                    )
    
    train.set_index(['id','step'], inplace=True)
    return train

X_train = process_json(train_json, True)
X_test = process_json(test_json, False)
print(X_train.shape, X_test.shape)

So we got 256800 = 2400\*107 rows. But since the measured values are there only for first 68 pairs, we will have NaN on the remaining columns for these entries. This is not cleaned here since the user can easily remove those with dropna if he wants only 68 entries or can fill with dummy values if he wants to use all 107. So we will have 2400\*39 = 96300 NaN values in these columns.

In [ ]:
X_train.isnull().any()

In [ ]:
X_train.isnull().sum()

In [ ]:
X_train.head()

In [ ]:
print(X_test[X_test.seq_length==107].shape, X_test[X_test.seq_length==130].shape)

Test set has 629\*107 = 67303 + 3005\*130 = 390650 entries corresponding to 3634 rows in test set.

In [ ]:
print(X_test.isnull().any())

There are no NaN entries as expected.

In [ ]:
X_test.head()

### Write to CSV file

In [ ]:
X_train.to_csv('train.csv')
X_test.to_csv('test.csv')

In [ ]:
!cp ../input/stanford-covid-vaccine/sample_submission.csv sample_submission.csv 

### Read back and verify shape and contents

In [ ]:
test_train = pd.read_csv('train.csv', index_col=[0,1])
print(X_train.dtypes)
print(test_train.dtypes)

Lets compare the columns of same type (float64 not compared as floating point numbers will not match due to precision)

In [ ]:
cols = ['sequence','structure','predicted_loop_type']
print(X_train.shape, test_train.shape, test_train[cols].equals(X_train[cols]))

In [ ]:
X_train.head()

In [ ]:
test_train.head()

In [ ]:
test_test = pd.read_csv('test.csv', index_col=[0,1])
print(X_test.dtypes)
print(test_test.dtypes)

In [ ]:
cols = ['sequence','structure','predicted_loop_type']
print(X_test.shape, test_test.shape, test_test[cols].equals(X_test[cols]))

In [ ]:
X_test.head()

In [ ]:
test_test.head()

# EOF